# Finger Individuation Session Analysis

This notebook demonstrates how to analyze session logs from the Leap Motion finger individuation game.

## Setup

In [ ]:
# Install dependencies if needed
# !pip install matplotlib pandas numpy

In [ ]:
import sys
sys.path.insert(0, '..')  # Add parent directory to path

from analysis import SessionAnalyzer, list_sessions, compare_sessions
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

%matplotlib inline
plt.style.use('seaborn-v0_8-whitegrid')  # Nice plot style
plt.rcParams['figure.dpi'] = 100

## 1. List Available Sessions

In [ ]:
# List all available session files (most recent first)
sessions = list_sessions('../session_logs')

## 2. Load a Session

In [ ]:
# Load the most recent session
analyzer = SessionAnalyzer()
analyzer.load_session(sessions[0])  # Change index to load different session

In [ ]:
# View session summary
summary = analyzer.get_summary()
for key, value in summary.items():
    if isinstance(value, float):
        print(f"{key:25s}: {value:.2f}")
    else:
        print(f"{key:25s}: {value}")

## 3. Session Overview Plot

This shows:
- Trial timeline with correct/incorrect markers
- Reaction time per trial
- Motion Leakage Ratio (MLR) per trial
- Accuracy breakdown by finger
- Score progression

In [ ]:
fig = analyzer.plot_session_overview(figsize=(14, 10))
plt.show()

## 4. Individual Trial Analysis

View detailed hand positions for a specific trial.

In [ ]:
# Plot trial 1 (change number to view different trials)
fig = analyzer.plot_trial(1)
plt.show()

In [ ]:
# Get detailed info for a trial
trial = analyzer.get_trial(1)
print(f"Trial {trial.number}:")
print(f"  Target: {trial.target_finger}")
print(f"  Pressed: {trial.pressed_finger}")
print(f"  Correct: {trial.is_correct}")
print(f"  Reaction Time: {trial.reaction_time_ms:.0f} ms")
print(f"  MLR: {trial.mlr:.4f}")
print(f"  Clean Trial: {trial.is_clean_trial}")
print(f"  Coupled Keypress: {trial.coupled_keypress}")

## 5. All Trials Sequence View

See all trials in a horizontal sequence - good for spotting patterns.

In [ ]:
fig = analyzer.plot_all_trials_sequence()
plt.show()

## 6. Confusion Matrix (Target vs Pressed)

Shows which fingers were confused with each other.

In [ ]:
fig = analyzer.plot_finger_heatmap()
plt.show()

## 7. DataFrame for Custom Analysis

Export to pandas DataFrame for further analysis.

In [ ]:
df = analyzer.to_dataframe()
df

In [ ]:
# Custom analysis example: Average MLR by finger
df.groupby('target')['mlr'].agg(['mean', 'std', 'count']).round(3)

In [ ]:
# Reaction time distribution
df['reaction_time_ms'].hist(bins=20, edgecolor='black')
plt.xlabel('Reaction Time (ms)')
plt.ylabel('Count')
plt.title('Reaction Time Distribution')
plt.axvline(df['reaction_time_ms'].mean(), color='red', linestyle='--', label=f'Mean: {df["reaction_time_ms"].mean():.0f}ms')
plt.legend()
plt.show()

## 8. Compare Multiple Sessions

In [ ]:
# Compare last 5 sessions
if len(sessions) >= 2:
    comparison_df = compare_sessions(sessions[:5])
    display(comparison_df[['session_id', 'total_trials', 'accuracy', 'clean_rate', 'avg_mlr', 'avg_reaction_time_ms', 'final_score']])
else:
    print("Need at least 2 sessions to compare")

In [ ]:
# Plot progress over sessions
if len(sessions) >= 2:
    comparison_df = compare_sessions(sessions[:10])  # Last 10 sessions
    comparison_df = comparison_df.iloc[::-1]  # Reverse to chronological order
    
    fig, axes = plt.subplots(2, 2, figsize=(12, 8))
    
    # Accuracy over sessions
    axes[0, 0].plot(comparison_df['accuracy'], 'b-o')
    axes[0, 0].set_ylabel('Accuracy (%)')
    axes[0, 0].set_title('Accuracy Over Sessions')
    axes[0, 0].set_xlabel('Session')
    
    # Clean trial rate
    axes[0, 1].plot(comparison_df['clean_rate'], 'g-o')
    axes[0, 1].set_ylabel('Clean Trial Rate (%)')
    axes[0, 1].set_title('Clean Trial Rate Over Sessions')
    axes[0, 1].set_xlabel('Session')
    
    # Average MLR
    axes[1, 0].plot(comparison_df['avg_mlr'], 'r-o')
    axes[1, 0].set_ylabel('Average MLR')
    axes[1, 0].set_title('Average MLR Over Sessions (lower = better)')
    axes[1, 0].set_xlabel('Session')
    
    # Reaction time
    axes[1, 1].plot(comparison_df['avg_reaction_time_ms'], 'm-o')
    axes[1, 1].set_ylabel('Avg Reaction Time (ms)')
    axes[1, 1].set_title('Average Reaction Time Over Sessions')
    axes[1, 1].set_xlabel('Session')
    
    plt.tight_layout()
    plt.show()

## 9. Export Data for External Tools

In [ ]:
# Export to CSV for use in R, Excel, etc.
# df.to_csv('session_analysis.csv', index=False)
# print("Exported to session_analysis.csv")